# Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from math import ceil 
from pprint import pprint

import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.options.display.max_columns=100 
pd.options.display.max_rows=100

# Funções 

In [ ]:
def find_missing_percent(data):
    """
    Retorna dataframe contendo o total de valores faltantes e porcentagem do total
    de valores faltantes da coluna.
    """
    miss_df = pd.DataFrame({'ColumnName':[],'TotalMissingVals':[],'PercentMissing':[]})
    for col in data.columns:
        sum_miss_val = data[col].isnull().sum()
        percent_miss_val = round((sum_miss_val/data.shape[0])*100,2)
        miss_df.loc[len(miss_df)] = dict(zip(miss_df.columns,[col,sum_miss_val,percent_miss_val]))
    return miss_df

In [ ]:
def calculate_outlier_percentage(series, threshold=1.5):
     """
     Função para calcular porcentagem de outliers para uma dada série
     """
     z_scores = np.abs((series - series.median()) / series.std())
     outliers = z_scores > threshold
     return (outliers.sum() / len(series)) * 100

In [ ]:
def create_multiple_boxplots(data_frame, columns_for_boxplot, titles=None, num_boxplots_per_row=2):
    # Calcular a quantidade;
    num_boxplots = len(columns_for_boxplot)
    num_rows = (num_boxplots + num_boxplots_per_row - 1) // num_boxplots_per_row

    # Criar os subplots
    fig = make_subplots(rows=num_rows, cols=num_boxplots_per_row, subplot_titles=titles)

    # Loop para ir montando todos os gráficos em boxplot
    for idx, column in enumerate(columns_for_boxplot):
        row_idx = idx // num_boxplots_per_row + 1
        col_idx = idx % num_boxplots_per_row + 1

        data = data_frame[column]
        box = go.Box(y=data, name=column)

        fig.add_trace(box, row=row_idx, col=col_idx)

    # Ajustando a forma
    fig.update_layout(height=300*num_rows, showlegend=False)

    # Plotar os gráficos
    fig.show()


In [ ]:
def plotar_distribuicoes(data_frame, columns_for_distribution, num_distributions_per_row=2):
    # Calcular a quantidade
    num_distributions = len(columns_for_distribution)
    num_rows = ceil(num_distributions / num_distributions_per_row)

    # Criar os subplots
    fig = make_subplots(rows=num_rows, cols=num_distributions_per_row)

    # Loop para ir montando todos os gráficos de distribuição
    for idx, column in enumerate(columns_for_distribution):
        dados = data_frame[column].dropna()  # Remover valores ausentes

        # Criar o gráfico de histograma
        histogram_data = go.Histogram(x=dados, nbinsx=30, name=f'Histograma - {column}')

        # Adicionar ao subplot
        fig.add_trace(histogram_data,
                      row=(idx // num_distributions_per_row) + 1, col=(idx % num_distributions_per_row) + 1)

    # Atualizar o layout com títulos e legendas adequadas
    for idx, column in enumerate(columns_for_distribution):
        row_idx = (idx // num_distributions_per_row) + 1
        col_idx = (idx % num_distributions_per_row) + 1

        # Adicionar título ao subplot
        fig.update_xaxes(title_text=f'{column}', row=row_idx, col=col_idx)
        fig.update_yaxes(title_text='Quantidade', row=row_idx, col=col_idx)  # Adicionar título ao eixo Y

    # Ajustando a forma
    fig.update_layout(height=300*num_rows, showlegend=False)  # Remover a legenda

    # Plotar os gráficos
    fig.show()

In [ ]:
def cria_fluxograma(df, origem, destino, intermediario, titulo, tam_fonte):
    df_temp1 = df.groupby([origem, intermediario])['order_status'].count().reset_index()
    df_temp1.columns = ['source', 'target', 'value']

    df_temp2 = df.groupby([intermediario, destino])['delivery_status'].count().reset_index()
    df_temp2.columns = ['source', 'target', 'value']

    if destino == 'order_status':
        df_temp2['target'] = df_temp2['target'].map({'CANCELED': 'Cancelado', 'FINISHED': 'Finalizado'})
    else:
        pass

    links = pd.concat([df_temp1, df_temp2], axis=0)

    unique_source_target = list(pd.unique(links[['source', 'target']].values.ravel('K')))

    mapping_dict = {k: v for v, k in enumerate(unique_source_target)}

    links['source'] = links['source'].map(mapping_dict)
    links['target'] = links['target'].map(mapping_dict)

    links_dict = links.to_dict(orient='list')

    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness = 20,
            line = dict(color = 'black', width = 0.5),
            label = unique_source_target,
            color = 'blue'
        ),
        link = dict(
            source = links_dict['source'],
            target = links_dict['target'],
            value = links_dict['value']
        )
    )])

    return fig.update_layout(title_text = titulo, font_size=tam_fonte)

In [ ]:
def drop_multiple_col(col_names_list, df): 
    '''
    AIM    -> Drop multiple columns based on their column names 
    
    INPUT  -> List of column names, df
    
    OUTPUT -> updated df with dropped columns 
    ------
    '''
    df.drop(col_names_list, axis=1, inplace=True)
    return df

In [ ]:
def find_correlated_columns(df, interval):
    """
    Encontra e exibe as correlações entre colunas de um DataFrame.

    Parâmetros:
    - df: DataFrame pandas
    - intervalo de correlação desejado (uma tupla de dois valores)

    Retorna:
    - Lista de tuplas representando pares de colunas correlacionadas.
    """
    correlation_matrix = df.corr(numeric_only=True)
    correlated_columns = []

    # Iterar sobre as combinações de colunas para encontrar correlações
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            corr = correlation_matrix.iloc[i, j]
            if interval[0] <= abs(corr) <= interval[1]:
                col1 = correlation_matrix.columns[i]
                col2 = correlation_matrix.columns[j]
                correlated_columns.append((col1, col2))
                print(f"Correlação entre {col1} e {col2}: {corr}")

    # Plotar um mapa de calor da matriz de correlação
    plt.figure(figsize=(20, 16))
    sns.heatmap(correlation_matrix, annot=True, cmap='cubehelix_r')
    plt.title('Matriz de Correlação')
    plt.xlabel('Variáveis')
    plt.ylabel('Variáveis')
    plt.show()

    return correlated_columns

In [ ]:
def correlacao_com_variavel_alvo(df, target_variable, nivel="forte", top_n=5):
    """
    Imprime as n features com as maiores correlações com uma variável alvo, com base no nível escolhido.

    Parâmetros:
    - df: DataFrame pandas.
    - target_variable: String, nome da variável alvo.
    - nivel: String que define o critério de correlação ("forte", "fraca", etc.).
    - top_n: Número inteiro, quantidade de features a serem impressas.

    Retorna:
    - Nenhum (imprime as correlações).
    """
    correlation_matrix = df.corr(numeric_only=True)

    # Filtra as correlações com base no nível escolhido
    if nivel.lower() == "forte":
        filtered_correlations = correlation_matrix[((correlation_matrix >= 0.7) & (correlation_matrix < 1.0)) | ((correlation_matrix <= -0.7) & (correlation_matrix > -1.0))]
    else:
        raise ValueError("Nível não suportado. Atualmente, apenas 'forte' é suportado.")

    # Filtra as correlações com a variável alvo
    correlations_with_target = filtered_correlations[target_variable].sort_values(ascending=False)

    # Pegar as n maiores correlações
    top_n_correlations = correlations_with_target.head(top_n)

    # Imprimir as n maiores correlações com a variável alvo
    print(f"As {top_n} maiores correlações com '{target_variable}' ({nivel}):")
    for feature, correlation in top_n_correlations.items():
        print(f"{feature}: {correlation}")


In [ ]:
def convert_format_to_datetime(dataframe, columns):
    """
    Convert specified columns in a DataFrame from 'DD/MM/YYYY HH:MM:SS PM' to 'YYYY-MM-DD HH:MM:SS' format.

    Parameters:
    - dataframe (pd.DataFrame): The DataFrame to modify.
    - columns (list): List of column names to convert.

    Returns:
    - pd.DataFrame: The modified DataFrame.
    """
    for column in columns:
        if column in dataframe.columns:
            # Convert the specified column to datetime with the original format
            dataframe[column] = pd.to_datetime(dataframe[column], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
        else:
            print(f"Column '{column}' not found in the DataFrame.")

    return dataframe

In [ ]:
def avalia_modelo(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = round(np.sqrt(mse),2)
    mae = round(mean_absolute_error(y_test,y_pred),2)
    
    return rmse, mae

# Limpeza dos Dados

Faremos a junção de todas as tabelas em uma só para facilitar a manipulação conjunta dos dados

In [ ]:
# Carregando as bases
channels = pd.read_csv('channels.csv')
deliveries = pd.read_csv('deliveries.csv')
drivers = pd.read_csv('drivers.csv')
hubs = pd.read_csv('hubs.csv')
orders = pd.read_csv('orders.csv')
payments = pd.read_csv('payments.csv')
stores = pd.read_csv('stores.csv')

# Fazendo a unificação
deliveries = pd.merge(left=drivers, right=deliveries, on='driver_id', how ='right')
stores = pd.merge(left=hubs, right=stores, on='hub_id', how ='right')
df = pd.merge(left=channels, right=orders, on='channel_id', how ='right')
df = pd.merge(left=payments, right=df, on='payment_order_id', how ='right')
df = pd.merge(left=deliveries, right=df, on='delivery_order_id', how ='right')
df = pd.merge(left=stores, right=df, on='store_id', how ='right')


In [ ]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)

In [ ]:
moment_order_columns = ['order_moment_created', 'order_moment_accepted','order_moment_ready', 
                        'order_moment_collected','order_moment_in_expedition', 'order_moment_delivering',
                        'order_moment_finished']

# Convertendo as colunas selecionadas para tipo datetime
df = convert_format_to_datetime(df, columns=moment_order_columns)

# Criando uma coluna com o tempo de entrega do pedido (essa é a variável que vai prevista) - se assemelha MUITO com order_metric_cycle_time
df['tempo_entrega'] = (df['order_moment_finished'] - df['order_moment_accepted']).dt.round('MIN')

In [ ]:
# Criando uma coluna de data com base nas colunas existentes
df.rename(columns = {'order_created_year':'year','order_created_month':'month','order_created_day':'day', 'order_created_hour': 'hour', 'order_created_minute': 'minute'}, inplace=True)
df['order_date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])

In [ ]:
miss_df = find_missing_percent(df)
display(miss_df[miss_df['PercentMissing']>0.0])
print("\n")
print(f"Número de colunas com valores faltantes:{str(miss_df[miss_df['PercentMissing']>0.0].shape[0])}")

In [ ]:
# Tirando as colunas com mais de 70% de dados faltantes
for coluna in miss_df.loc[miss_df['PercentMissing'] > 70]['ColumnName']:
    df.drop(columns=coluna, inplace=True)

Com base na análise da funcionalidade de cada coluna e na porcentagem de dados faltantes, vamos focar somente nas que podem nos trazer mais resultados e excluir as mais problemáticas

Observação: A coluna criada 'tempo_entrega' que utiliza o 'order_moment_accepted' e 'order_moment_finished' apresenta MUITA SEMELHANÇA com a coluna 'order_metric_cycle_time'.

Quando os valores são muito diferentes, isso se dá porque o 'order_moment_finished', que corresponde ao momento que o estabelecimento finalizou o pedido, está muito depois do tempo correspondente à entrega em si, o que denota uma inconsistência nos dados

Devido à isso, a coluna 'tempo_entrega' será desconsiderada para futuras análises e a métrica utilizada pra fazer as previsões em etapas posteriores será 'order_metric_cycle_time'

In [ ]:
colunas_descart = ['hub_id', 'hub_latitude', 'hub_longitude', 'store_id', 'store_latitude', 'store_longitude', 'driver_id',
                   'delivery_id', 'delivery_order_id','payment_id', 'payment_order_id', 'channel_id', 'order_id',
                   'hour', 'minute', 'day', 'month', 'year', 'tempo_entrega']

df = drop_multiple_col(colunas_descart, df)

'''Separando dados numéricos e categóricos '''
numeric_cols = df.select_dtypes(['float','int']).columns
categoric_cols = df.select_dtypes('object').columns

df_numeric = df[numeric_cols]
df_categoric = df[categoric_cols]

In [ ]:
miss_df = find_missing_percent(df)
'''Displays columns with missing values'''
display(miss_df[miss_df['PercentMissing']>0.0])
print("\n")
print(f"Número de colunas com valores faltantes:{str(miss_df[miss_df['PercentMissing']>0.0].shape[0])}")

In [ ]:
numeric_cols = df.select_dtypes(['float','int']).columns
for feature in numeric_cols:
    qtd_outliers = calculate_outlier_percentage(df[feature]).round(2)
    print(f'{qtd_outliers} % | {feature}' )

In [ ]:
#Fazendo boxplots, para visualizar os outliers
create_multiple_boxplots(df, df_numeric.columns,num_boxplots_per_row=5)

In [ ]:
#Plotando distribuições
plotar_distribuicoes(df, df_numeric.columns)

In [ ]:
linhas_antes = df.shape[0]

# removendo outliers de delivery_distance_meters
outliers = df[df['delivery_distance_meters'] > 10000].index
df.drop(outliers, inplace=True)

# removendo outliers de payment_amount
outliers = df[df['payment_amount'] > 300].index
df.drop(outliers, inplace=True)

# removendo outliers de payment_fee
outliers = df[df['payment_fee'] > 10].index
df.drop(outliers, inplace=True)

# removendo outliers de order_amount
outliers = df[df['order_amount'] > 300].index
df.drop(outliers, inplace=True)

# removendo outliers de order_delivery_fee
outliers = df[df['order_delivery_fee'] > 27].index
df.drop(outliers, inplace=True)

# removendo outliers de order_delivery_cost
outliers = df[df['order_delivery_cost'] > 15].index
df.drop(outliers, inplace=True)

# removendo outliers de order_metric_collected_time
outliers = df[(df['order_metric_collected_time'] > 7) | (df['order_metric_walking_time'] < 0)].index
df.drop(outliers, inplace=True)

# removendo outliers de order_metric_paused_time
outliers = df[ (df['order_metric_paused_time'] < 0) | (df['order_metric_paused_time'] > 20)].index
df.drop(outliers, inplace=True)

# removendo outliers de order_metric_production_time
outliers = df[df['order_metric_production_time'] > 45].index
df.drop(outliers, inplace=True)

# removendo outliers de order_metric_walking_time
outliers = df[(df['order_metric_walking_time'] > 10) | (df['order_metric_walking_time'] < 0)].index
df.drop(outliers, inplace=True)

# removendo outliers de order_metric_expediton_speed_time
outliers = df[df['order_metric_expediton_speed_time'] > 20].index
df.drop(outliers, inplace=True)

# removendo outliers de order_metric_transit_time
outliers = df[df['order_metric_transit_time'] > 45].index
df.drop(outliers, inplace=True)

# removendo outliers de order_metric_cycle_time
outliers = df[df['order_metric_cycle_time'] > 90].index
df.drop(outliers, inplace=True)

# removendo outliers de tempo_entrega que excedam 1 dia para realizar a entrega
#outliers = df[df['tempo_entrega'] > pd.Timedelta(days=1)].index
#df.drop(outliers, inplace=True)

df.reset_index(drop=True, inplace=True)

linhas_depois = df.shape[0]
restante = round((100* linhas_depois / linhas_antes), 2)
print(f'total de linhas retiradas: {linhas_antes - linhas_depois} (restam {linhas_depois} linhas que equivalem a {restante}% da base inicial)')

In [ ]:
df[df['payment_status'].eq('AWAITING')]

In [ ]:
# removendo AWAITING de 'paymen_status' porque são somente 10 valores e 8 deles não apresentam a métrica de interesse 
outliers = df[df['payment_status'].eq('AWAITING')].index
df.drop(outliers, inplace=True)

df.reset_index(drop=True, inplace=True)

In [ ]:
# Plotar distribuições
plotar_distribuicoes(df, df_numeric.columns)

In [ ]:
round(df.describe().T,2)

# Análise Exploratória

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])
df_date = df.copy()

# Criando colunas para facilitar análises
df_date['minute'] = df['order_date'].dt.minute
df_date['hour'] = df['order_date'].dt.hour
df_date['day'] = df['order_date'].dt.day
df_date['month'] = df['order_date'].dt.month
df_date['year'] = df['order_date'].dt.year
df_date['weekday'] = df['order_date'].dt.weekday # Domingo é 0

In [ ]:
# Agrupando o DataFrame pelas cidades e seus estados correspondentes
grouped_data = df.groupby(['hub_city', 'hub_state']).size().reset_index(name='Count')

# Criando o treemap
fig = px.treemap(grouped_data, path=['hub_state', 'hub_city'], values='Count',
                 title='Cidades em que Hack Eats está presente')
fig.show()

In [ ]:
cria_fluxograma(df, origem='hub_name', destino='order_status', intermediario='hub_state', 
                titulo='Fluxo Cancelamento das Distribuições por Estados', tam_fonte=10)

In [ ]:
cria_fluxograma(df_date, origem='hub_state', destino='order_status', intermediario='hour', 
                titulo='Fluxo Cancelamento dos Estados por Hora do Dia', tam_fonte=10)

In [ ]:
df_temp1 = df.groupby(['hub_state', 'driver_modal'])['order_status'].count().reset_index()
df_temp1.columns = ['source', 'target', 'value']
df_temp1['target'] = df_temp1['target'].map({'BIKER': 'Ciclista', 'MOTOBOY': 'Motociclista'})

df_temp2 = df.groupby(['driver_modal', 'order_status'])['delivery_status'].count().reset_index()
df_temp2.columns = ['source', 'target', 'value']
df_temp2['source'] = df_temp2['source'].map({'BIKER': 'Ciclista', 'MOTOBOY': 'Motociclista'})
df_temp2['target'] = df_temp2['target'].map({'CANCELED': 'Cancelado', 'FINISHED': 'Finalizado'})

links = pd.concat([df_temp1, df_temp2], axis=0)

unique_source_target = list(pd.unique(links[['source', 'target']].values.ravel('K')))

mapping_dict = {k: v for v, k in enumerate(unique_source_target)}

links['source'] = links['source'].map(mapping_dict)
links['target'] = links['target'].map(mapping_dict)

links_dict = links.to_dict(orient='list')

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness = 20,
        line = dict(color = 'black', width = 0.5),
        label = unique_source_target,
        color = 'blue'
    ),
    link = dict(
        source = links_dict['source'],
        target = links_dict['target'],
        value = links_dict['value']
    )
)])

fig.update_layout(title_text = 'Fluxo Cancelamento dos Estados por Tipo de Entregador', font_size=15)

In [ ]:
x_plot = 'hour'
plt.figure(figsize=(10,6))
sns.countplot(x=x_plot, data=df_date, palette='viridis', hue='order_status')
plt.xlabel('Horário do Dia')
plt.ylabel('Pedidos')
plt.title('Quantidade de Pedidos por Horário do Dia')
plt.show()

In [ ]:
x_plot = 'weekday'

plt.figure(figsize=(10,6))
sns.countplot(x=x_plot, data=df_date[~df_date['delivery_status'].eq('DELIVERING')], hue='delivery_status')
plt.xlabel('Dia da Semana (Início no Domingo)')
plt.ylabel('Pedidos')
plt.title('Pedidos por Dias da Semana')
plt.show()

In [ ]:
x_plot = 'weekday'

plt.figure(figsize=(10,6))
#sns.countplot(x=x_plot, data=df, hue=x_plot, legend=False, palette='viridis')
sns.countplot(x=x_plot, data=df_date, hue='hub_state')
plt.xlabel('Dia da Semana (Início no Domingo)')
plt.ylabel('Pedidos')
plt.title('Quantidade de Pedidos por Dia da Semana em Diferentes Estados')
plt.show()

In [ ]:
x_plot = 'month'

plt.figure(figsize=(10,6))
sns.countplot(x=x_plot, data=df_date)
plt.xlabel('Mês do Ano')
plt.ylabel('Pedidos')
plt.title('Quantidade de Pedidos por Mês do Ano')
plt.show()

In [ ]:
y_plot = 'order_metric_cycle_time'
x_plot = 'delivery_distance_meters'

plt.figure(figsize=(10,6))
sns.scatterplot(x=x_plot,y=y_plot, data=df)
plt.xlabel('Tempo de Entrega')
plt.ylabel('Distância de Entrega')
plt.title('Distância por Tempo de Entrega')
plt.show()

In [ ]:
fig = px.histogram(df, x='driver_type', y='order_metric_cycle_time', histfunc='avg')
fig.update_layout(
    title='Tempo médio de entrega por tipo de motorista',
    xaxis_title='Tipo de motorista',
    yaxis_title='Tempo médio de entrega'
)
fig.show()

In [ ]:
fig = px.histogram(df, x='store_name', y='order_metric_cycle_time', histfunc='avg')
fig.update_layout(
    title='Tempo médio de entrega por loja',
    xaxis_title='Loja',
    yaxis_title='Tempo médio de entrega'
)
fig.show()

In [ ]:
fig = px.histogram(df, x='payment_method', y='order_metric_cycle_time', histfunc='avg')
fig.update_layout(
    title='Tempo médio de entrega por método de pagamento',
    xaxis_title='Método de pagamento',
    yaxis_title='Tempo médio de entrega'
)
fig.show()

In [ ]:
fig = px.histogram(df, x='channel_name', y='order_metric_cycle_time', histfunc='avg')
fig.update_layout(
    title='Tempo médio de entrega por canal',
    xaxis_title='Canais',
    yaxis_title='Tempo médio de entrega'
)
fig.show()

In [ ]:
y_plot = 'order_metric_cycle_time'
x_plot = 'weekday'

plt.figure(figsize=(10,6))
sns.barplot(x=x_plot,y=y_plot, data=df_date)
plt.xlabel('Dia da Semana')
plt.ylabel('Tempo Médio de Entrega')
plt.title('Tempo Médio de Entrega por Dia da Semana')
plt.show()

In [ ]:
y_plot = 'order_metric_cycle_time'
x_plot = 'hour'

plt.figure(figsize=(10,6))
sns.barplot(x=x_plot,y=y_plot, data=df_date)
plt.xlabel('Horário do dia')
plt.ylabel('Tempo Médio de Entrega')
plt.title('Tempo Médio de Entrega por Horário do dia')
plt.show()

In [ ]:
x_plot = 'order_metric_cycle_time'
y_plot = 'hub_state'

plt.figure(figsize=(10,6))
sns.barplot(x=x_plot,y=y_plot, data=df_date, orient='h')
plt.ylabel('Estado')
plt.xlabel('Tempo Médio de Entrega')
plt.title('Tempo Médio de Entrega por Estado')
plt.show()

In [ ]:
x_plot = 'order_metric_cycle_time'
y_plot = 'hub_name'

plt.figure(figsize=(10,6))
sns.barplot(x=x_plot,y=y_plot, data=df_date, orient='h')
plt.ylabel('Estado')
plt.xlabel('Tempo Médio de Entrega')
plt.title('Tempo Médio de Entrega por Local de Distribuição')
plt.show()

In [ ]:
y_plot = 'order_metric_cycle_time'
x_plot = 'driver_modal'

plt.figure(figsize=(10,6))
sns.barplot(x=x_plot,y=y_plot, data=df_date)
plt.xlabel('DRIVER_MODAL')
plt.ylabel('Tempo Médio de Entrega')
plt.title('Tempo Médio de Entrega por DRIVER_MODAL')
plt.show()

In [ ]:
x_plot = 'order_metric_cycle_time'
y_plot = 'channel_type'

plt.figure(figsize=(10,6))
sns.barplot(x=x_plot,y=y_plot, data=df_date, orient='h')
plt.ylabel('Tipo de Canal')
plt.xlabel('Tempo Médio de Entrega')
plt.title('Tempo Médio de Entrega por Tipo de Canal')
plt.show()

In [ ]:
x_plot = 'order_metric_cycle_time'
y_plot = 'channel_type'

plt.figure(figsize=(10,6))
sns.barplot(x=x_plot,y=y_plot, data=df_date, orient='h', hue='hub_state')
plt.ylabel('Tipo de Canal')
plt.xlabel('Tempo Médio de Entrega')
plt.title('Tempo Médio de Entrega por Tipo de Canal')
plt.show()

# Processamento dos Dados

Os valores outliers já foram tratados na etapa de limpeza dos dados, agora cabe lidar com os valores nulos e com o tratamento das colunas para viabilizar a utilização dos modelos de Machine Learning

In [ ]:
# Mantendo somente os pedidos que não foram cancelados para que possamos analisar os tempos de entrega
df = df[~df['order_status'].eq('CANCELED')]

df.reset_index(drop=True, inplace=True)

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['order_moment_accepted'] = pd.to_datetime(df['order_moment_accepted'])

# Criando colunas para facilitar análises
# Colunas para o tempo de realização do pedido
df['minute_rea'] = df['order_date'].dt.minute
df['hour_rea'] = df['order_date'].dt.hour
df['day_rea'] = df['order_date'].dt.day
df['month_rea'] = df['order_date'].dt.month
df['year_rea'] = df['order_date'].dt.year
df['weekday_rea'] = df['order_date'].dt.weekday # Domingo é 0

# Colunas para o tempo de aceitação do pedido
df['minute_ac'] = df['order_moment_accepted'].dt.minute
df['hour_ac'] = df['order_moment_accepted'].dt.hour
df['day_ac'] = df['order_moment_accepted'].dt.day
df['month_ac'] = df['order_moment_accepted'].dt.month
df['year_ac'] = df['order_moment_accepted'].dt.year
df['weekday_ac'] = df['order_moment_accepted'].dt.weekday # Domingo é 0

# Tempo para aceitar o pedido
df['dif_tempo'] = df['order_moment_accepted'] - df['order_date']
df['min_para_aceitar'] = df['dif_tempo'].apply(lambda x: round(x.total_seconds() / 60 ,2))

df.drop(columns=['order_date', 'order_moment_accepted', 'dif_tempo'], axis=1, inplace=True)

In [ ]:
# removendo colunas que não serão úteis para prever o tempo de entrega, já que não teremos informações delas no momento do pedido
col_del = ['order_status', 'payment_status', 'delivery_status', 'order_moment_ready', 'order_moment_collected',
       'order_moment_in_expedition', 'order_moment_delivering',
       'order_moment_finished', 'order_metric_collected_time',
       'order_metric_paused_time', 'order_metric_production_time',
       'order_metric_walking_time', 'order_metric_expediton_speed_time',
       'order_metric_transit_time']

# tirando os ano já que só possuo dados de 2021
col_del = col_del + ['year_rea', 'year_ac'] 
df.drop(col_del, axis=1, inplace=True)

In [ ]:
find_missing_percent(df)

In [ ]:
linhas_antes = df.shape[0]
# Removendo pedidos sem o tempo de entrega 'order_metric_cycle_time'
df = df[~df['order_metric_cycle_time'].isnull()]

# Preenchendo valores faltantes do store_plan_price para poder dividir em colunas por OneHotEncoding
df['store_plan_price'].fillna(-1, inplace=True)

# REMOVENDO TODAS AS LINHAS COM VALORES NULOS 
df.dropna(axis=0, inplace=True)

df.reset_index(drop=True, inplace=True)

linhas_depois = df.shape[0]
print(f'apaguei {linhas_antes - linhas_depois} linhas e restam {round(100*linhas_depois / linhas_antes)}% do df')

In [ ]:
# Vamos listar as features que vamos utilizar
input_cols_categoric = ['hub_state','channel_type', 'store_plan_price']
input_cols_numeric = ['delivery_distance_meters', 'payment_amount', 'payment_fee', 'order_amount',  
                    'minute_rea', 'hour_rea', 'day_rea', 'month_rea',
                     'weekday_rea', 'minute_ac', 'hour_ac','day_ac', 'month_ac', 
                     'weekday_ac', 'min_para_aceitar']

features = input_cols_categoric + input_cols_numeric

target = 'order_metric_cycle_time'

In [ ]:
# Dividindo os dados 
X = df[features]  
y = df[target]  

# Divide em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=999
)

In [ ]:
# Tratando as variáveis categóricas 

encoder = ce.OneHotEncoder(cols=input_cols_categoric, handle_unknown='ignore')

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

# Modelagem

In [ ]:
# Parâmetros do modelo com melhor previsão do tempo de entrega
parametros= {'base_score': None,
            'booster': 'gbtree',
            'callbacks': None,
            'colsample_bylevel': None,
            'colsample_bynode': None,
            'colsample_bytree': None,
            'device': None,
            'early_stopping_rounds': None,
            'enable_categorical': False,
            'eval_metric': None,
            'feature_types': None,
            'gamma': 0.3,
            'grow_policy': None,
            'importance_type': None,
            'interaction_constraints': None,
            'learning_rate': 0.05,
            'max_bin': None,
            'max_cat_threshold': None,
            'max_cat_to_onehot': None,
            'max_delta_step': None,
            'max_depth': 12,
            'max_leaves': None,
            'min_child_weight': None,
            'monotone_constraints': None,
            'multi_strategy': None,
            'n_estimators': 900,
            'n_jobs': None,
            'num_parallel_tree': None,
            'objective': 'reg:squarederror',
            'random_state': 0,
            'reg_alpha': 0.4,
            'reg_lambda': 0.1,
            'sampling_method': None,
            'scale_pos_weight': None,
            'subsample': 0.9,
            'tree_method': None,
            'validate_parameters': None,
            'verbosity': None}

In [ ]:
# Treinando o modelo com os parâmetros
XGB_model = XGBRegressor(**parametros)

# Fazendo uma validação cruzada para verificar os resultados no dados de treino
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
result = cross_val_score(XGB_model, X_train, y_train, cv = kfold, scoring='neg_root_mean_squared_error')

In [ ]:
# Resultados da validação cruzada
print(f'K-Fold RMSE Scores: {result}')
print(f'Mean RMSE for Cross-Validation K-Fold: {result.mean():.2f}')

In [ ]:
# Avaliando o modelo com os dados de teste, de forma avaliar a eficiência dele em dados novos (assim como em uma situação real)
XGB_model.fit(X_train, y_train)
rmse, mae = avalia_modelo(XGB_model, X_test, y_test)

In [ ]:
print(f'RMSE: {rmse}  |  MAE: {mae}')

In [ ]:
# Importância de cada coluna no modelo
feature_imp = pd.Series(XGB_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

# Criando um gráfico de barras
sns.set_palette("mako_r")
_ = plt.figure(figsize=(10, 6))
_ = sns.barplot(x=feature_imp, y=feature_imp.index)
_ = plt.xlabel("Feature Importance Score")
_ = plt.ylabel("Features")
_ = plt.title("Visualizing Important Features")